In [1]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.2/645.2 kB 6.1 MB/s eta 0:00:00


In [5]:
!rm -rf /content/VisDrone2019-DET-val


In [7]:
%mkdir /content/dataset
%cd /content/dataset

/content/dataset


In [ ]:
!unzip /content/drive/MyDrive/258-DL/HW2-ObjectDetection/datasets/visdrone/VisDrone2019-DET-val.zip -d /content/dataset/

In [ ]:
!yolo task=detect \
mode=predict \
source="/content/drive/MyDrive/258-DL/HW2-ObjectDetection/datasets/images" \
imgsz=1280 \
save=True \
save_conf=True \
save_txt=True \
name=yolov8n_v8_50e_infer1280 \
hide_labels=False

In [16]:
import json
import os
from PIL import Image

def create_coco_annotation(visdrone_dir):
    # Initialize COCO dataset structure
    coco_dataset = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # Define your categories here
    categories = {
        0: "ignore",
        1: "pedestrian",
        2: "people",
        3: "bicycle",
        4: "car",
        5: "van",
        6: "truck",
        7: "tricycle",
        8: "awning-tricycle",
        9: "bus",
        10: "motor",
        11: "others"
    }

    # Adding categories to COCO dataset
    for id, name in categories.items():
        coco_dataset["categories"].append({"id": id, "name": name})

    # Process each image and its annotations
    annotation_id = 1
    for image_file in os.listdir(os.path.join(visdrone_dir, "images")):
        # Extracting image name without extension
        image_name = os.path.splitext(image_file)[0]

        # Load image to get width and height
        image_path = os.path.join(visdrone_dir, "images", image_file)
        with Image.open(image_path) as img:
            width, height = img.size

        # Add image information to COCO dataset
        coco_dataset["images"].append({
            "id": image_name,  # Using image name without extension as ID
            "width": width,
            "height": height,
            "file_name": image_file
        })

        # Read corresponding annotation file
        annotation_file = image_file.replace('.jpg', '.txt').replace('.png', '.txt')
        with open(os.path.join(visdrone_dir, "annotations", annotation_file)) as f:
            for line in f:
                # Parse VisDrone annotation line
                x, y, w, h, category_id, _, _, _ = map(int, line.split(','))

                # Add annotation to COCO dataset
                coco_dataset["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_name,  # Using image name without extension as ID
                    "category_id": category_id,
                    "bbox": [x, y, w, h],
                    "area": w * h,
                    "iscrowd": 0
                })
                annotation_id += 1

    # Save COCO dataset to a JSON file
    with open('visdrone_coco.json', 'w') as f:
        json.dump(coco_dataset, f, indent=2)

# Example usage
create_coco_annotation('/content/dataset/VisDrone2019-DET-val/')


In [ ]:
from ultralytics import YOLO
model = YOLO()
# model = YOLO("/content/ultra/runs/detect/train3/weights/best.pt")
# results = model.predict("/content/drive/MyDrive/258-DL/HW2-ObjectDetection/datasets/visdrone/VisDrone2019-DET-val/images")
results = model.predict("/content/dataset/VisDrone2019-DET-val/images")
# results = model.predict("/content/ultra/datasets/VisDrone/VisDrone2019-DET-val/images/0000271_01401_d_0000380.jpg")
# results = model.predict("/content/images")

In [11]:
import json

json_list = []

for result in results:
  # print(result)
  base_name = os.path.basename(result.path)
  name, ext = os.path.splitext(base_name)
  image_id = str(name)
  # print(image_id)

  output = []
  for box in result.boxes:
      # print(box)

      x1, y1, x2, y2 = [
        round(x,2) for x in box.xyxy[0].tolist()
      ]
      # class_id = box.cls[0].item()
      class_id = int(box.cls[0].item())
      prob = round(box.conf[0].item(), 3)
      output.append([
        x1, y1, x2, y2, int(class_id), prob
      ])
  # print(output)

  json_object = {
      "image_id": image_id,
      "category_id": class_id,
      "bbox": [x1, y1, x2, y2],
      "score": prob
  }
  json_list.append(json_object)

print(json_list)

filename = '/content/inference-coco.json'

with open(filename, 'w') as file:
    json.dump(json_list, file, indent=4)



[{'image_id': '0000001_02999_d_0000005', 'category_id': 2, 'bbox': [619.18, 279.15, 712.4, 347.42], 'score': 0.263}, {'image_id': '0000001_03499_d_0000006', 'category_id': 2, 'bbox': [1038.49, 159.26, 1122.96, 196.46], 'score': 0.314}, {'image_id': '0000001_03999_d_0000007', 'category_id': 2, 'bbox': [838.26, 467.31, 885.0, 502.08], 'score': 0.345}, {'image_id': '0000001_04527_d_0000008', 'category_id': 2, 'bbox': [1545.86, 796.4, 1579.73, 828.66], 'score': 0.264}, {'image_id': '0000001_05249_d_0000009', 'category_id': 0, 'bbox': [1256.03, 983.08, 1293.54, 1041.77], 'score': 0.259}, {'image_id': '0000001_05499_d_0000010', 'category_id': 2, 'bbox': [477.69, 409.72, 555.86, 470.36], 'score': 0.297}, {'image_id': '0000001_05999_d_0000011', 'category_id': 0, 'bbox': [1547.86, 994.01, 1578.24, 1056.69], 'score': 0.252}, {'image_id': '0000001_07999_d_0000012', 'category_id': 2, 'bbox': [813.67, 405.13, 877.96, 439.08], 'score': 0.254}, {'image_id': '0000001_08414_d_0000013', 'category_id': 2

In [ ]:
#/content/drive/MyDrive/258-DL/HW2-ObjectDetection/datasets/val-coco-single.json

In [ ]:
# %cd /content/drive/MyDrive/258-DL/HW2-ObjectDetection/datasets/visdrone/
# !unzip /content/drive/MyDrive/258-DL/HW2-ObjectDetection/datasets/visdrone/VisDrone2019-DET-val.zip

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [17]:
# annFile = "/content/val-ground-truth.json"
annFile = "/content/dataset/visdrone_coco.json"
# annFile = "/content/instances_val.json"
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!


In [18]:
resFile = "/content/inference-coco.json"
# resFile = "/content/instances_val2014_fakebbox100_results.json"
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!


In [19]:
imgIds=sorted(cocoGt.getImgIds())
# imgIds=imgIds
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,"bbox")
# cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()


Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.64s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [15]:
!yolo detect val model=yolov8n.pt

WARNING ⚠️ 'data' is missing. Using default 'data=coco8.yaml'.
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset 'coco8.yaml' images not found ⚠️, missing path '/content/datasets/coco8/images/val'
100% 433k/433k [00:00<00:00, 30.5MB/s]
Unzipping /content/datasets/coco8.zip to /content/datasets/coco8...: 100% 25/25 [00:00<00:00, 3168.57file/s]
Dataset download success ✅ (1.4s), saved to /content/datasets

100% 755k/755k [00:00<00:00, 37.6MB/s]
val: Scanning /content/datasets/coco8/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<00:00, 365.59it/s]
val: New cache created: /content/datasets/coco8/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:01<00:00,  1.91s/it]
                   all          4         17      0.621      0.833      0.888       0.63
                person     

In [16]:
model = YOLO('yolov8n.pt')
# %cd /content

/content


In [19]:
results = model.train(epochs=5, imgsz=640, save=True, optimizer='SGD', val=True,  momentum=0.8)

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=5, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fals

In [20]:
!yolo detect val model='/content/runs/detect/train/weights/best.pt'

WARNING ⚠️ 'data' is missing. Using default 'data=coco8.yaml'.
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:01<00:00,  1.19s/it]
                   all          4         17      0.908      0.541      0.777      0.533
                person          4         10      0.942        0.3      0.485      0.229
                   dog          4          1          1          0      0.497      0.272
                 horse          4          2          1      0.947      0.995      0.698
              elephant          4          2          1          0      0.695      0.208
              umbrella          4          1      0.758          1      0.995      0